### ライブラリをインポート

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import japanize_matplotlib
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as MSE

### CSVファイルを読み込む

In [7]:
# train_tmp.csv,test_new.csv、condition_new.csvを読み込む
train_tmp = pd.read_csv('train_tmp.csv')
test_new = pd.read_csv('test_new.csv')
condition_new = pd.read_csv('condition_new.csv')

# 既存CSVファイルを再読み込み
sample = pd.read_csv('sample_submit.csv',header=None,names=["id","y"])
stadium = pd.read_csv('stadium.csv')

# 確認
print(train_tmp.shape)
print(test_new.shape)
print(condition_new.shape)
print(sample.shape)
print(stadium.shape)
print(sample.info())

(1952, 48)
(351, 10)
(2304, 31)
(313, 2)
(59, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 313 entries, 0 to 312
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      313 non-null    int64  
 1   y       313 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 5.0 KB
None


In [81]:
train_tmp['flg'] = 0
test_tmp['flg'] = 1

#train_tmpとtest_newを結合する
data_all = pd.concat([train_tmp,test_tmp])

data_all.head()

,id,y,year,stage,match,gameday,time,home,away,stadium,...,address,capa,month,week,match_num,tv_num,di,last_match,forlan,flg
0,13994,18250.0,2012,Ｊ１,第１節第１日,03/10(土),14:04,ベガルタ仙台,鹿島アントラーズ,ユアテックスタジアム仙台,...,宮城県仙台市泉区七北田字柳78,19694,3,土,1,4,2.508,0,0,0
1,13995,24316.0,2012,Ｊ１,第１節第１日,03/10(土),14:04,名古屋グランパス,清水エスパルス,豊田スタジアム,...,愛知県豊田市千石町7-2,40000,3,土,1,4,5.332,0,0,0
2,13996,17066.0,2012,Ｊ１,第１節第１日,03/10(土),14:04,ガンバ大阪,ヴィッセル神戸,万博記念競技場,...,大阪府吹田市千里万博公園5-2,21000,3,土,1,4,4.633,0,0,0
3,13997,29603.0,2012,Ｊ１,第１節第１日,03/10(土),14:06,サンフレッチェ広島,浦和レッズ,エディオンスタジアム広島,...,広島県広島市安佐南区大塚西5-1-1,50000,3,土,1,4,5.928,0,0,0
4,13998,25353.0,2012,Ｊ１,第１節第１日,03/10(土),14:04,コンサドーレ札幌,ジュビロ磐田,札幌ドーム,...,北海道札幌市豊平区羊ヶ丘1,39232,3,土,1,4,7.200,0,0,0


### データ前処理

In [8]:
# test_newとcondetion_newをマージする
test_tmp = pd.merge(test_new,condition_new,on='id',how='left')

# test_tmpの確認
print(test_tmp.head())
print(test_tmp.shape)

      id  year stage    match   gameday   time       home       away  \
0  15822  2014    Ｊ１  第１８節第１日  08/02(土)  19:04     ベガルタ仙台  大宮アルディージャ   
1  15823  2014    Ｊ１  第１８節第１日  08/02(土)  18:34   鹿島アントラーズ  サンフレッチェ広島   
2  15824  2014    Ｊ１  第１８節第１日  08/02(土)  19:04      浦和レッズ    ヴィッセル神戸   
3  15825  2014    Ｊ１  第１８節第１日  08/02(土)  19:03      柏レイソル   川崎フロンターレ   
4  15827  2014    Ｊ１  第１８節第１日  08/02(土)  19:03  アルビレックス新潟     セレッソ大阪   

          stadium                                tv  ...  away_02  away_03  \
0    ユアテックスタジアム仙台              スカパー！／スカパー！プレミアムサービス  ...    中村　北斗    今井　智基   
1  県立カシマサッカースタジアム              スカパー！／スカパー！プレミアムサービス  ...     塩谷　司    千葉　和彦   
2     埼玉スタジアム２００２  スカパー！／スカパー！プレミアムサービス／ＮＨＫ　ＢＳ１／テレ玉  ...    河本　裕之    増川　隆洋   
3        日立柏サッカー場              スカパー！／スカパー！プレミアムサービス  ...   小宮山　尊信    實藤　友紀   
4  デンカビッグスワンスタジアム              スカパー！／スカパー！プレミアムサービス  ...    藤本　康太    丸橋　祐介   

  away_04  away_05 away_06 away_07   away_08 away_09 away_10  away_11  
0   横山　知伸    高橋　祥平   橋本　晃司

In [12]:
# test_tmpとstadiumをマージする
test_tmp = pd.merge(test_tmp,stadium,left_on='stadium',right_on='name',how='left')

# test_tmpの確認
print(test_tmp.head())
print(test_tmp.shape)

# 重複しているカラムを削除
test_tmp = test_tmp.drop(columns=['name'])

# 確認
print(test_tmp.shape)
print(test_tmp.info())

(351, 42)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 0 to 350
Data columns (total 42 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           351 non-null    int64  
 1   year         351 non-null    int64  
 2   stage        351 non-null    object 
 3   match        351 non-null    object 
 4   gameday      351 non-null    object 
 5   time         351 non-null    object 
 6   home         351 non-null    object 
 7   away         351 non-null    object 
 8   stadium      351 non-null    object 
 9   tv           351 non-null    object 
 10  home_score   351 non-null    int64  
 11  away_score   351 non-null    int64  
 12  weather      351 non-null    object 
 13  temperature  351 non-null    float64
 14  humidity     351 non-null    object 
 15  referee      351 non-null    object 
 16  home_team    351 non-null    object 
 17  home_01      351 non-null    object 
 18  home_02      351 non-null    object 
 19

### 関数でデータを補正する

In [13]:
# gamedayから月を抜き出してmonthカラムに設定する
def get_month(x):
    return int(x[0:2])

# gamedayから曜日を抜き出してweekカラムに設定する
def get_week(x):
    return x[6:7]

# matchから節数を抜き出してmatch_numカラムに設定する
def get_match(x):
    pos_s = 0
    pos_e = 0
    pos_s = x.find("第")
    pos_e = x.find("節")
    if pos_s != -1 and pos_e != -1:
        pos_s += 1
        return int(x[pos_s:pos_e])
    else:
        return 0

# humidityの"%"を削除して小数に変換する
def get_humidity(x):
    return float(x.rstrip('%')) / 100

# train_tmpにtv_numを追加して放送局数を設定する
def get_tvnum(x):
    return len(x.split('／'))

test_tmp['tv_num'] = test_tmp['tv'].apply(get_tvnum)

# 関数を呼び出す
test_tmp['month'] = test_tmp['gameday'].apply(get_month)
test_tmp['week'] = test_tmp['gameday'].apply(get_week)
test_tmp['match_num'] = test_tmp['match'].apply(get_match)
test_tmp['humidity'] = test_tmp['humidity'].apply(get_humidity)
test_tmp['tv_num'] = test_tmp['tv'].apply(get_tvnum)

# 確認する
print(test_tmp['month'].unique())
print(test_tmp['week'].unique())
print(test_tmp['match_num'].unique())
print(test_tmp['humidity'].head().unique())
print(test_tmp['tv_num'].unique())

[ 8  9 10 11 12]
['土' '火' '日' '水' '月' '金']
[18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42]
[0.7  0.65 0.58 0.76 0.68]


In [14]:
# test_tmpの欠損値を確認する
print(test_tmp.isnull().sum())

id             0
year           0
stage          0
match          0
gameday        0
time           0
home           0
away           0
stadium        0
tv             0
home_score     0
away_score     0
weather        0
temperature    0
humidity       0
referee        0
home_team      0
home_01        0
home_02        0
home_03        0
home_04        0
home_05        0
home_06        0
home_07        0
home_08        0
home_09        0
home_10        0
home_11        0
away_team      0
away_01        0
away_02        0
away_03        0
away_04        0
away_05        0
away_06        0
away_07        0
away_08        0
away_09        0
away_10        0
away_11        0
address        0
capa           0
month          0
week           0
match_num      0
dtype: int64


In [16]:
# チーム名の確認
print(test_tmp['home'].value_counts())
print(test_tmp['away'].value_counts())


カマタマーレ讃岐       10
栃木ＳＣ           10
京都サンガF.C.      10
ザスパクサツ群馬       10
ギラヴァンツ北九州       9
湘南ベルマーレ         9
ファジアーノ岡山        9
ロアッソ熊本          9
松本山雅ＦＣ          9
ベガルタ仙台          9
アビスパ福岡          9
大分トリニータ         9
ジェフユナイテッド千葉     9
東京ヴェルディ         9
Ｖ・ファーレン長崎       9
水戸ホーリーホック       9
カターレ富山          9
鹿島アントラーズ        9
横浜ＦＣ            9
モンテディオ山形        9
サガン鳥栖           9
清水エスパルス         9
アルビレックス新潟       9
ガンバ大阪           9
横浜Ｆ・マリノス        9
徳島ヴォルティス        9
大宮アルディージャ       9
浦和レッズ           8
柏レイソル           8
ジュビロ磐田          8
ＦＣ岐阜            8
川崎フロンターレ        8
セレッソ大阪          8
ヴァンフォーレ甲府       8
名古屋グランパス        8
ヴィッセル神戸         8
ＦＣ東京            8
サンフレッチェ広島       8
愛媛ＦＣ            8
コンサドーレ札幌        8
Name: home, dtype: int64
ジュビロ磐田         10
ＦＣ岐阜           10
コンサドーレ札幌       10
愛媛ＦＣ           10
ロアッソ熊本          9
アビスパ福岡          9
ジェフユナイテッド千葉     9
水戸ホーリーホック       9
東京ヴェルディ         9
Ｖ・ファーレン長崎       9
カターレ富山          9
大分トリニータ         9
横浜ＦＣ            9
湘南ベルマーレ         9
松本山

In [18]:
# 最後の２節であることを表すlast_matchカラムを作成する
test_tmp['last_match'] = 0
test_tmp['last_match'][((test_tmp['match_num'] == 33) | (test_tmp['match_num'] == 34)) & (test_tmp['stage'] == 'Ｊ１')] = 1
test_tmp['last_match'][((test_tmp['match_num'] == 41) | (test_tmp['match_num'] == 42)) & (test_tmp['stage'] == 'Ｊ２')] = 1

# last_matchカラムが作成できたか確認して、last_matchに該当する(1になっている)試合数を変数chkに入力してください。
print(test_tmp['last_match'].value_counts())


0    311
1     40
Name: last_match, dtype: int64


C:\Users\cax68\AppData\Local\Temp\ipykernel_14708\1330854909.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tmp['last_match'][((test_tmp['match_num'] == 33) | (test_tmp['match_num'] == 34)) & (test_tmp['stage'] == 'Ｊ１')] = 1
C:\Users\cax68\AppData\Local\Temp\ipykernel_14708\1330854909.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tmp['last_match'][((test_tmp['match_num'] == 41) | (test_tmp['match_num'] == 42)) & (test_tmp['stage'] == 'Ｊ２')] = 1


In [19]:
# 2014年のawayがセレッソ大阪の試合を1にそれ以外を0にしたforlanカラムを作成
test_tmp['forlan'] = 0
test_tmp['forlan'][(test_tmp['year'] == 2014) & (test_tmp['away']=="セレッソ大阪")] = 1

# forlanカラムが作成できたかを確認
print(test_tmp['forlan'].value_counts())


0    342
1      9
Name: forlan, dtype: int64


C:\Users\cax68\AppData\Local\Temp\ipykernel_14708\301285794.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tmp['forlan'][(test_tmp['year'] == 2014) & (test_tmp['away']=="セレッソ大阪")] = 1


### モデル作成

In [22]:
# train_tmp,train_dummy,train_dummy_1,train_dummy_2をCSVから読み込む。
train_tmp = pd.read_csv('train_tmp.csv')
train_dummy = pd.read_csv('train_dummy.csv')
train_dummy_1 = pd.read_csv('train_dummy_1.csv')
train_dummy_2 = pd.read_csv('train_dummy_2.csv')

In [26]:
# 最後の２節であることを表すlast_matchカラムを作成する
train_tmp['last_match'] = 0
train_tmp['last_match'][((train_tmp['match_num'] == 33) | (train_tmp['match_num'] == 34)) & (train_tmp['stage'] == 'Ｊ１')] = 1
train_tmp['last_match'][((train_tmp['match_num'] == 41) | (train_tmp['match_num'] == 42)) & (train_tmp['stage'] == 'Ｊ２')] = 1

# last_matchカラムが作成できたか確認して、last_matchに該当する(1になっている)試合数を変数chkに入力してください。
print(train_tmp['last_match'].value_counts())


0    1872
1      80
Name: last_match, dtype: int64


C:\Users\cax68\AppData\Local\Temp\ipykernel_14708\3346447374.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_tmp['last_match'][((train_tmp['match_num'] == 33) | (train_tmp['match_num'] == 34)) & (train_tmp['stage'] == 'Ｊ１')] = 1
C:\Users\cax68\AppData\Local\Temp\ipykernel_14708\3346447374.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_tmp['last_match'][((train_tmp['match_num'] == 41) | (train_tmp['match_num'] == 42)) & (train_tmp['stage'] == 'Ｊ２')] = 1


In [27]:
# 2014年のawayがセレッソ大阪の試合を1にそれ以外を0にしたforlanカラムを作成
train_tmp['forlan'] = 0
train_tmp['forlan'][(train_tmp['year'] == 2014) & (train_tmp['away']=="セレッソ大阪")] = 1

# forlanカラムが作成できたかを確認
print(train_tmp['forlan'].value_counts())


0    1944
1       8
Name: forlan, dtype: int64


C:\Users\cax68\AppData\Local\Temp\ipykernel_14708\3175350377.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_tmp['forlan'][(train_tmp['year'] == 2014) & (train_tmp['away']=="セレッソ大阪")] = 1


In [94]:
y_1 = train_dummy['y']
print(y_1)


0       18250.0
1       24316.0
2       17066.0
3       29603.0
4       25353.0
         ...   
1947     5776.0
1948     3865.0
1949     6420.0
1950     3711.0
1951     2221.0
Name: y, Length: 1952, dtype: float64


In [93]:
# last_matchとforlanカラムを加える
#select_columns = ['id','y','capa','week','home','away','month','stage','tv_num','weather','last_match','forlan','flg']
#data_dummy = pd.get_dummies(data_all[select_columns],drop_first=True)

#train_dummy = data_dummy[data_dummy['flg'] == 0]
#test_dummy = data_dummy[data_dummy['flg'] == 1]

print(train_dummy['y'].value_counts())
print(test_dummy['y'].info())

8741.0    3
9237.0    2
3230.0    2
6516.0    2
2781.0    2
         ..
6585.0    1
6144.0    1
3173.0    1
3564.0    1
2221.0    1
Name: y, Length: 1849, dtype: int64
<class 'pandas.core.series.Series'>
Int64Index: 351 entries, 0 to 350
Series name: y
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 5.5 KB
None


In [95]:
# last_matchとforlanカラムを加える
#select_columns = ['id','y','capa','week','home','away','month','stage','tv_num','weather','last_match','forlan','flg']
#data_dummy = pd.get_dummies(data_all[select_columns],drop_first=True)


y_1 = train_dummy['y']
train_dummy = train_dummy.drop(columns=['id','y'])

X_train,X_test,y_train,y_test = train_test_split(train_dummy, y_1, random_state = 1234)

# モデルの作成
lr = LinearRegression()
# モデルの学習
lr.fit(X_train,y_train)

#訓練データの予測・精度評価
y_train_pred = lr.predict(X_train)
rmse_train = np.sqrt(MSE(y_train,y_train_pred))
print(rmse_train)

#テストデータの予測・精度評価
y_test_pred = lr.predict(X_test)
rmse_test = np.sqrt(MSE(y_test,y_test_pred))
print(rmse_test)

2921.645060291492
3180.2025205572463


In [96]:
# 評価データで予測する
#select_columns = ['id','y','capa','week','home','away','month','stage','tv_num','weather','last_match','forlan']
#test_dummy = pd.get_dummies(test_dat[select_columns],drop_first=True)

test_dummy = test_dummy.drop(columns=['id','y'])

test_pred = lr.predict(test_dummy)

print(test_pred)

[13191.18853595 18204.48298351 36240.12109047 12319.42791905
 36282.35485531 14092.08016631  2584.70372664 13738.07801106
 13460.10073756 22254.70750386 27361.2609876   9888.55818945
 22519.48377554 13321.72501597 14543.7147324  12119.30909629
 37414.1845346  10097.66073985 23243.99943093 16048.30040505
  5851.48074002 12974.30139654 27798.8262545   9863.30352969
 30022.88933839 15897.82051114 22188.02992167 16140.98385183
 17487.3559659  11880.89898513 13825.17017684 27024.91842343
 16157.22580174 35753.57167038  8863.62302977 15436.66999841
 13438.48926096 14933.17557499 14608.01808501 22651.82617634
  8883.62558128 12676.16454386 23449.16568449 14552.71954929
 28879.48629243 11195.04871301 28254.56612273 23879.94640264
 20464.96068891 15490.64528981 37169.64314028 16801.85110427
 11735.47588372 26217.67797062 17469.44261544 16167.67194118
  5139.2297812  11163.0354584  17375.41032111 11289.48272277
 22392.65495442 30234.72043028 10885.52118473 31542.55145151
 19277.58603791 25300.91

In [103]:
sample['y'] = test_pred[0:313]
sample.to_csv('sample.csv',header=False,index=False)